## Apache License 2.0 YoloV4 Pseudo Labeling + OOF Evaluation

Thanks everyone who shared about EfficientDet、Yolov4、Pseudo Labeling and OOF, let me finished this notebook.

References:  

Last original : https://www.kaggle.com/hawkey/yolov5-pseudo-labeling-oof-evaluation

Awesome original Pseudo Labeling notebook : https://www.kaggle.com/nvnnghia/yolov5-pseudo-labeling  
Evaluation Script : https://www.kaggle.com/pestipeti/competition-metric-details-script  
OOF-Evaluation : https://www.kaggle.com/shonenkov/oof-evaluation-mixup-efficientdet  
Bayesian Optimization (though failed to improve my results) : https://www.kaggle.com/shonenkov/bayesian-optimization-wbf-efficientdet  

Yolov4 : https://www.kaggle.com/zacchaeus/train-yolov4-pytorch

EfficientDet : https://www.kaggle.com/shonenkov/inference-efficientdet



In [ ]:
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import shutil as sh
import torch
#import sys
from glob import glob
import time
from pathlib import Path
TIME_LIMIT = 6.9*60*60 # about 9 hours runtime limit
#KERNEL_START_TIME = time.time()
##Pesudo
Origin = False
p_HF = True
p_ROT = False
eimg_size = 1024
pesudo_thres = 0.4
pesudo_iou = 0.35
pesudo_score_threshold = 0
###########################
img_size = 1024
NMS_IOU_THR = 0.6
NMS_CONF_THR = 0.35
# WBF
best_iou_thr = 0.35
best_skip_box_thr = 0.35

# Box conf threshold
best_final_score = 0
best_score_threshold = 0.1

EPO = 8 #15-start #80-last

WEIGHTS = "../input/yolov4-pytorch/last_trainF5_onlyfix.weights"

PESUDO_WEIGHTS = '../input/effdet-cutmix-augmix-result/best-checkpoint_d7.bin'

CONFIG = '../input/yolov4-pytorch/cd53s_self_1class.cfg'

OPTIMIZER = '../input/yolov4-pytorch/last_optimizer.pth'

is_TEST = len(os.listdir('../input/global-wheat-detection/test/'))>11

is_AUG = False
is_ROT = True

VALIDATE = False

PSEUDO = True
####Predict
Yolov4 = True
Effdet = True
GET_PESUDO = True
# For OOF evaluation
csv_file = '../input/yolov4-pytorch/newtrain.csv'
#csv_file = '../input/global-wheat-detection/train.csv'
marking = pd.read_csv(csv_file)

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)
max_label_numbers = 0
pesudo_result = []

In [ ]:
####
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain,DetBenchEval
from effdet.efficientdet import HeadNet

In [ ]:
sys.path.insert(0, "../input/pytorchyolov4/pytorch-YOLOv4")
#sys.path.insert(0, "../input/newyolov4-pytorch/pytorch-YOLOv4")
from tool.darknet2pytorch import Darknet
from tool.utils import *
from tool.torch_utils import *
import argparse
import cv2
!cp -r ../input/pytorchyolov4/pytorch-YOLOv4/* .
#!cp -r ../input/newyolov4-pytorch/pytorch-YOLOv4/* .
sys.path.insert(0, "../input/weightedboxesfusion")

def process_det(index, det, score_threshold=NMS_CONF_THR):
    scores = det[index][:, 5].copy()
    det = det[index][:, :4].copy()
    bboxes = np.zeros((det.shape))
    bboxes[:, 0] = ((det[:, 0] - det[:, 2] / 2) * 1024).astype(int)
    bboxes[:, 1] = ((det[:, 1] - det[:, 3] / 2) * 1024).astype(int)
    bboxes[:, 2] = ((det[:, 0] + det[:, 2] / 2) * 1024).astype(int)
    bboxes[:, 3] = ((det[:, 1] + det[:, 3] / 2) * 1024).astype(int)
    bboxes = (bboxes).clip(min = 0, max = 1024).astype(int)
    
    indexes = np.where(scores>score_threshold)
    bboxes = bboxes[indexes]
    scores = scores[indexes]
    return bboxes, scores

In [ ]:
def convertTrainLabel(max_label_numbers):
    df = pd.read_csv(csv_file)
    bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        df[column] = bboxs[:,i]
    df.drop(columns=['bbox'], inplace=True)
    df['x1'] = df['x'] + df['w']
    df['y1'] = df['y'] + df['h']
    df['x_center'] = df['x'] + df['w']/2
    df['y_center'] = df['y'] + df['h']/2
    df['classes'] = 0
    from tqdm.auto import tqdm
    import shutil as sh
    df = df[['image_id','x', 'y', 'w', 'h','x1','y1','x_center','y_center','classes']]
    
    index = list(set(df.image_id))
    source = 'train'
    if True:
        for fold in [0]:
            val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
            for name,mini in tqdm(df.groupby('image_id')):
                os.makedirs('convertor', exist_ok=True)
                sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/{}.jpg'.format(name))
                if name in val_index:
                    path2save = 'convertor/val.txt'
                else:
                    path2save = 'convertor/train.txt'
                with open(path2save, 'a') as f:
                    f.write(f'{name}.jpg')
                    row = mini[['x','y','x1','y1','classes']].astype(int).values
                    if row.shape[0]>max_label_numbers:
                        max_label_numbers = row.shape[0]
                    # row = row/1024
                    row = row.astype(str)
                    for j in range(len(row)):
                        text = ' '+','.join(row[j])
                        f.write(text)
                    f.write('\n')
    return max_label_numbers

In [ ]:
from ensemble_boxes import *

def run_wbf(boxes, scores, image_size=1024, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size)
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

In [ ]:
# validate

import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple


@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area


@jit(nopython=True)
def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

@jit(nopython=True)
def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


@jit(nopython=True)
def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

    
# Numba typed list!
iou_thresholds = numba.typed.List()

for x in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]:
    iou_thresholds.append(x)
def validate():
    """hyper parameters"""
    use_cuda = True
    truth = {}
    DATA_ROOT_PATH = '../input/global-wheat-detection/train'
    with open('convertor/val.txt', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            data = line.split(" ")
            truth[data[0]] = []
            for i in data[1:]:
                truth[data[0]].append([int(j) for j in i.split(',')])

    image_ids = [image_id.split('.')[0] for image_id in list(truth.keys())]
    #image_ids = image_ids[0:10]
    
    weights = 'checkpoints/last.pt'
    Effdet = False
    if not os.path.exists(weights):
        Effdet = True
        model = Darknet(CONFIG)
        model.load_weights(WEIGHTS)
        print('OLD WEIGHT')
    else:
        model = torch.load(weights)

    if use_cuda:
        model.cuda()
        
    if Effdet:
        net = get_valnet_file(PESUDO_WEIGHTS)
        
    model.eval()
    num_classes = model.num_classes
    class_names = 'wheat'
    
    results = []
    
    for image_id in image_ids:
        enboxes = []
        enscores = []
        
        img = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg')
        img = cv2.resize(img, (model.width, model.height))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if Yolov4:
            if is_ROT:
                for i in range(4):
                    img1 = TTAImage(img, i)
                    img1 = img1.transpose(2, 0, 1)      
                    img1 = torch.from_numpy(img1).float().div(255.0).unsqueeze(0)
                    img1 = img1.cuda()
                    img1 = torch.autograd.Variable(img1)
                    with torch.no_grad():
                        output = model(img1)
                        boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                        det = np.array(boxes)
                        boxes, scores = process_det(0, det, NMS_CONF_THR)
                        for _ in range(3-i):
                            boxes = rotBoxes90(boxes, *img.shape[:2])  
                        enboxes.append(boxes)
                        enscores.append(scores) 
            with torch.no_grad():
                img1 = img.transpose(2, 0, 1)      
                img1 = torch.from_numpy(img1).float().div(255.0).unsqueeze(0)
                img1 = img1.cuda()
                img1 = torch.autograd.Variable(img1)
                output = model(img1)
                boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                det = np.array(boxes)
                boxes, scores = process_det(0, det, NMS_CONF_THR)
                enboxes.append(boxes)
                enscores.append(scores) 
            
        if Effdet:
            img1 = img.transpose(2, 0, 1)
            img1 = torch.from_numpy(img1).cuda()
            img1 =  img1.float()  # uint8 to fp16/32
            img1 /= 255.0
            if p_HF:
                with torch.no_grad():
                    imgs = torch.stack((img1, )).float().cuda()
                    for tta_transform in tta_transforms:
                        det = net(tta_transform.batch_augment(imgs.clone()), torch.tensor([1]*imgs.shape[0]).float().cuda())

                        for i in range(imgs.shape[0]):
                            boxes = det[i].detach().cpu().numpy()[:,:4]    
                            scores = det[i].detach().cpu().numpy()[:,4]
                            indexes = np.where(scores > pesudo_thres)[0]
                            boxes = boxes[indexes]
                            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                            boxes = tta_transform.deaugment_boxes(boxes.copy())
                            enboxes.append(boxes)
                            enscores.append(scores) 
   
        records = marking[marking['image_id'] == image_id]
        gtboxes = records[['x', 'y', 'w', 'h']].values
        gtboxes = gtboxes.astype(np.int32).clip(min=0, max=1024)
        gtboxes[:, 2] = gtboxes[:, 0] + gtboxes[:, 2]
        gtboxes[:, 3] = gtboxes[:, 1] + gtboxes[:, 3]
            
        result = {
            'image_id': image_id,
            'pred_enboxes': enboxes, # xyx2y2
            'pred_enscores': enscores,
            'gt_boxes': gtboxes, # xyx2y2
        }
        results.append(result)
        
    return results

def calculate_final_score(all_predictions, iou_thr, skip_box_thr, score_threshold):
    final_scores = []
    for i in range(len(all_predictions)):
        gt_boxes = all_predictions[i]['gt_boxes'].copy()
        enboxes = all_predictions[i]['pred_enboxes'].copy()
        enscores = all_predictions[i]['pred_enscores'].copy()
        image_id = all_predictions[i]['image_id']
        
        pred_boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=iou_thr, skip_box_thr=skip_box_thr)    
        pred_boxes = pred_boxes.astype(np.int32).clip(min=0, max=1024)

        indexes = np.where(scores>score_threshold)
        pred_boxes = pred_boxes[indexes]
        scores = scores[indexes]

        image_precision = calculate_image_precision(gt_boxes, pred_boxes,thresholds=iou_thresholds,form='pascal_voc')
        final_scores.append(image_precision)

    return np.mean(final_scores)

def show_result(sample_id, preds, gt_boxes):
    sample = cv2.imread(f'../input/global-wheat-detection/train/{sample_id}.jpg', cv2.IMREAD_COLOR)
    sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for pred_box in preds:
        cv2.rectangle(
            sample,
            (pred_box[0], pred_box[1]),
            (pred_box[2], pred_box[3]),
            (220, 0, 0), 2
        )

    for gt_box in gt_boxes:    
        cv2.rectangle(
            sample,
            (gt_box[0], gt_box[1]),
            (gt_box[2], gt_box[3]),
            (0, 0, 220), 2
        )

    ax.set_axis_off()
    ax.imshow(sample)
    ax.set_title("RED: Predicted | BLUE - Ground-truth")

In [ ]:
# Bayesian Optimize

from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
from skopt.space import Categorical, Integer, Real

def log(text):
    print(text)

def optimize(space, all_predictions, n_calls=10):
    @use_named_args(space)
    def score(**params):
        
        log('-'*10)
        log(params)
        final_score = calculate_final_score(all_predictions, **params)
        log(f'final_score = {final_score}')
        log('-'*10)
        return -final_score

    return gp_minimize(func=score, dimensions=space, n_calls=n_calls)

# EFFD_PESUDO

In [ ]:
from glob import glob
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import path
img_size = 1024
def get_valid_transforms():
    return A.Compose([
        A.Resize(height=img_size, width=img_size, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.0)

DATA_ROOT_PATH = '../input/global-wheat-detection/test'

class DatasetRetriever_test(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        im0 = image
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id, im0

    def __len__(self) -> int:
        return self.image_ids.shape[0]

dataset = DatasetRetriever_test(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)
data_loader = DataLoader(
        dataset,
        batch_size=2,
        shuffle=False,
        num_workers=4,
        drop_last=False
    )

In [ ]:
from itertools import product
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = img_size

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError
class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [1,3]]
        res_boxes[:, [1,3]] = boxes[:, [2,0]]
        return res_boxes
class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

In [ ]:
model_struct = 'tf_efficientdet_d7'
def get_valnet_file(Get_Path):
    config = get_efficientdet_config(model_struct)
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = img_size
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(Get_Path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    #gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

def makePseudolabel():
    """hyper parameters"""
    use_cuda = True
    max_label_numbers = 0
    DATA_ROOT_PATH = '../input/global-wheat-detection/test'
    image_ids = [path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]
    
    model = Darknet(CONFIG)
    model.load_weights(WEIGHTS)

    if use_cuda:
        model.cuda()
        
    if Effdet:
        net = get_valnet_file(PESUDO_WEIGHTS)
        
    model.eval()
    num_classes = model.num_classes
    class_names = 'wheat'
    
    count = 0
    pesudo_result = []
    for image_id in image_ids:
        enboxes = []
        enscores = []
        try:
            img = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg')
            img = cv2.resize(img, (model.width, model.height))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if Yolov4:
                if is_ROT:
                    for i in range(4):
                        img1 = TTAImage(img, i)
                        img1 = img1.transpose(2, 0, 1)      
                        img1 = torch.from_numpy(img1).float().div(255.0).unsqueeze(0)
                        img1 = img1.cuda()
                        img1 = torch.autograd.Variable(img1)
                        with torch.no_grad():
                            output = model(img1)
                            boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                            det = np.array(boxes)
                            if det.size !=0:
                                boxes, scores = process_det(0, det, NMS_CONF_THR)
                                for _ in range(3-i):
                                    boxes = rotBoxes90(boxes, *img.shape[:2])  
                                enboxes.append(boxes)
                                enscores.append(scores) 
                with torch.no_grad():
                    img1 = img.transpose(2, 0, 1)      
                    img1 = torch.from_numpy(img1).float().div(255.0).unsqueeze(0)
                    img1 = img1.cuda()
                    img1 = torch.autograd.Variable(img1)
                    output = model(img1)
                    boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                    det = np.array(boxes)
                    if det.size !=0:
                        boxes, scores = process_det(0, det, NMS_CONF_THR)
                        enboxes.append(boxes)
                        enscores.append(scores) 

            if Effdet:
                img1 = img.transpose(2, 0, 1)
                img1 = torch.from_numpy(img1).cuda()
                img1 =  img1.float()  # uint8 to fp16/32
                img1 /= 255.0
                if p_HF:
                    with torch.no_grad():
                        imgs = torch.stack((img1, )).float().cuda()
                        for tta_transform in tta_transforms:
                            det = net(tta_transform.batch_augment(imgs.clone()), torch.tensor([1]*imgs.shape[0]).float().cuda())

                            for i in range(imgs.shape[0]):
                                boxes = det[i].detach().cpu().numpy()[:,:4]    
                                scores = det[i].detach().cpu().numpy()[:,4]
                                indexes = np.where(scores > pesudo_thres)[0]
                                boxes = boxes[indexes]
                                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                                boxes = tta_transform.deaugment_boxes(boxes.copy())
                                enboxes.append(boxes)
                                enscores.append(scores) 

            boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)    
            boxes = boxes.astype(np.int32).clip(min=0, max=1024)
            #boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            #boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            indices = scores >= best_score_threshold
            boxes = boxes[indices]
            scores = scores[indices]
            lineo = ''
            for box in boxes:
                x, y, x1, y1 = box
                lineo += ' %d,%d,%d,%d,0'%(x, y, x1, y1)
            os.makedirs('convertor', exist_ok=True)
            if len(lineo)>0:
                with open('convertor/train.txt', 'a') as f:
                    f.write(f'{image_id}.jpg')
                    f.write(lineo)
                    f.write('\n')
                sh.copy("../input/global-wheat-detection/test/{}.jpg".format(image_id),'convertor/{}.jpg'.format(image_id))
                result = {
                    'image_id': image_id,
                    'box': enboxes,
                    'score': enscores
                } 
                if boxes.shape[0]>max_label_numbers:
                    max_label_numbers = boxes.shape[0]
                pesudo_result.append(result)
                print(path,image_id)
        except:
            boxes = []
            scores = []
            print('one error')
        
    return pesudo_result, max_label_numbers
    #q.put(pesudo_result)
    #.put(max_label_numbers)

In [ ]:
if PSEUDO or is_TEST:
    max_label_numbers = convertTrainLabel(max_label_numbers)
if is_TEST and PSEUDO:
    """
    import multiprocessing
    q = multiprocessing.Queue()
    p = multiprocessing.Process(target=makePseudolabel, args=(q,))
    p.start()
    p.join()
    pesudo_result = q.get()
    pesudo_label_numbers = q.get()
    #
     """  
    pesudo_result, pesudo_label_numbers = makePseudolabel()
    if pesudo_label_numbers > max_label_numbers:
        max_label_numbers = pesudo_label_numbers
else:
    pesudo_result = []
#os.listdir('convertor/')
#os.remove('convertor/train.txt')

In [ ]:
if is_TEST:
    if PSEUDO:
        #remain_times = int( TIME_LIMIT - (time.time() - KERNEL_START_TIME) )0
        remain_times = 60
        !python train.py -TRAIN_EPOCHS {EPO} -cfgfile {CONFIG} -op {OPTIMIZER} -dir /kaggle/working/convertor -pretrained {WEIGHTS} -train_label_path convertor/train.txt -val_label_path convertor/val.txt -optimizer radam -iou-type ciou -l 0.0001 -g 0 -classes 1 -maxboxes {max_label_numbers+10} -time_limit {remain_times} 
    else:
        pass
else:
    pass
    #!python train.py --weights {WEIGHTS} --img 1024 --batch 2 --epochs 1 --data {DATA} --cfg {CONFIG}

In [ ]:
if VALIDATE and is_TEST:
    all_predictions = validate()
    
    # Bayesian Optimization: worse results.
    '''
    space = [
        Real(0, 1, name='iou_thr'),
        Real(0.25, 1, name='skip_box_thr'),
        Real(0, 1, name='score_threshold'),
    ]

    opt_result = optimize(
        space, 
        all_predictions,
        n_calls=50,
    )
    
    best_final_score = opt_result.fun
    best_iou_thr = opt_result.x[0]
    best_skip_box_thr = opt_result.x[1]
    best_score_threshold = opt_result.x[2]


    print('-'*13 + 'WBF' + '-'*14)
    print("[Baseline score]", calculate_final_score(all_predictions, 0.6, 0.43, 0))
    print(f'[Best Iou Thr]: {best_iou_thr:.3f}')
    print(f'[Best Skip Box Thr]: {best_skip_box_thr:.3f}')
    print(f'[Best Score Thr]: {best_score_threshold:.3f}')
    print(f'[Best Score]: {best_final_score:.4f}')
    print('-'*30)
    
    '''
    
    for score_threshold in tqdm(np.arange(0.1, 0.25, 0.01), total=np.arange(0.1, 0.25, 0.01).shape[0]):
        final_score = calculate_final_score(all_predictions, best_iou_thr, best_skip_box_thr, score_threshold)
        if final_score > best_final_score:
            best_final_score = final_score
            best_score_threshold = score_threshold

    print('-'*30)
    print(f'[Best Score Threshold]: {best_score_threshold}')
    print(f'[OOF Score]: {best_final_score:.4f}')
    print('-'*30)
    

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

def detect_cv2(pesudo_result):
    """hyper parameters"""
    use_cuda = True
    DATA_ROOT_PATH = '../input/global-wheat-detection/test'
    image_ids = [path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]
    
    weights = 'checkpoints/last.pt'
    Effdet = False
    if not os.path.exists(weights):
        Effdet = True
        model = Darknet(CONFIG)
        model.load_weights(WEIGHTS)
        print('OLD WEIGHT')
    else:
        #ckpt = torch.load(weights)
        #model = Darknet(CONFIG)
        #model.load_state_dict(ckpt)
        model = torch.load(weights)
    if (len(pesudo_result)>0) and (GET_PESUDO==True):
        print('get pesudo label')
        
    if use_cuda:
        model.cuda()
        
    if Effdet:
        net = get_valnet_file(PESUDO_WEIGHTS)
        
    model.eval()
    num_classes = model.num_classes
    class_names = 'wheat'
    
    results = []
    fig, ax = plt.subplots(5, 2, figsize=(30, 70))
    count = 0

    for image_id in image_ids:
        enboxes = []
        enscores = []
        try:
            img = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg')
            h, w = img.shape[:2]
            img = cv2.resize(img, (model.width, model.height))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if Yolov4:
                if is_ROT:
                    for i in range(4):
                        img1 = TTAImage(img, i)
                        with torch.no_grad():
                            img1 = img1.transpose(2, 0, 1)      
                            img1 = torch.from_numpy(img1).float().unsqueeze(0).cuda()
                            img1 /= 255.0
                            img1 = torch.autograd.Variable(img1)
                            output = model(img1)
                            boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                            det = np.array(boxes)
                            boxes, scores = process_det(0, det, NMS_CONF_THR)
                            for _ in range(3-i):
                                boxes = rotBoxes90(boxes, *img.shape[:2])  
                            enboxes.append(boxes)
                            enscores.append(scores) 
                with torch.no_grad():
                    img1 = img.transpose(2, 0, 1)      
                    img1 = torch.from_numpy(img1).float().unsqueeze(0).cuda()
                    img1 /= 255.0
                    img1 = torch.autograd.Variable(img1)
                    output = model(img1)
                    boxes = utils.post_processing(img1, NMS_CONF_THR, NMS_CONF_THR, output)
                    det = np.array(boxes)
                    boxes, scores = process_det(0, det, NMS_CONF_THR)
                    enboxes.append(boxes)
                    enscores.append(scores) 


            if Effdet:
                img1 = img.transpose(2, 0, 1)
                img1 = torch.from_numpy(img1).cuda()
                img1 =  img1.float()  # uint8 to fp16/32
                img1 /= 255.0
                if p_HF:
                    with torch.no_grad():
                        imgs = torch.stack((img1, )).float().cuda()
                        for tta_transform in tta_transforms:
                            det = net(tta_transform.batch_augment(imgs.clone()), torch.tensor([1]*imgs.shape[0]).float().cuda())

                            for i in range(imgs.shape[0]):
                                boxes = det[i].detach().cpu().numpy()[:,:4]    
                                scores = det[i].detach().cpu().numpy()[:,4]
                                indexes = np.where(scores > pesudo_thres)[0]
                                boxes = boxes[indexes]
                                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                                boxes = tta_transform.deaugment_boxes(boxes.copy())
                                enboxes.append(boxes)
                                enscores.append(scores)

            if (len(pesudo_result)>0) and (GET_PESUDO==True):  
                if next((inx  for (inx,dic) in enumerate(pesudo_result) if dic['image_id']==image_id), None) is not None:
                    enboxes.extend(pesudo_result[next((inx  for (inx,dic) in enumerate(pesudo_result) if dic['image_id']==image_id), None)]['box'])
                    enscores.extend(pesudo_result[next((inx  for (inx,dic) in enumerate(pesudo_result) if dic['image_id']==image_id), None)]['score']) 
            boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)    
            boxes = (boxes/1024*h).astype(np.int32).clip(min=0, max=h)  
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            indices = scores >= best_score_threshold
            boxes = boxes[indices]
            scores = scores[indices]

            if (count<10) and (is_TEST==False):
                img_ = img  # BGR
                for box, score in zip(boxes,scores):
                    cv2.rectangle(img_, (box[0], box[1]), (box[2]+box[0], box[3]+box[1]), (220, 0, 0), 2)
                    cv2.putText(img_, '%.2f'%(score), (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                ax[count%5][count//5].imshow(img_)
                count+=1
        except:
            boxes = []
            scores = []
            print('one error')
            pass
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        } 
        results.append(result)
    return results 

In [ ]:
results = detect_cv2(pesudo_result)
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()

In [ ]:
!rm -rf convertor